In [1]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 2 Aug 27 07:17 /dev/video2
crw-rw----+ 1 root video 81, 1 Aug 27 07:17 /dev/video1


In [2]:
from flask import Flask, Response
from ultralytics import YOLO
import cv2

# YOLO 모델 불러오기 (best.pt or .engine 가능)
model = YOLO("yolo.onnx", task="pose")

c_index = 1 #Camera index can be changed

cap = cv2.VideoCapture(c_index) 
assert cap.isOpened(), "Camera open failed."

app = Flask(__name__)

def gen_frames():
    while True:
        ret, frame = cap.read() 
        if not ret:
            break

        # YOLO inference
        results = model.predict(source=frame, imgsz=640, verbose=False)
        annotated = results[0].plot()  # draw inference results

        # JPEG encoding
        ret, buffer = cv2.imencode('.jpg', annotated)
        if not ret:
            continue #If encoding is failed, skip

        # MJPEG streaming: MJPEG = Motion JPEG
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=8000, debug=False, use_reloader=False)
    finally:
        cap.release()
        cv2.destroyAllWindows()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.55.1:8000
Press CTRL+C to quit


Loading best.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider


192.168.55.100 - - [27/Aug/2025 10:26:54] "GET / HTTP/1.1" 200 -
